In [133]:
import sqlparse
import pandas as pd
import pprint
from sqlparse.tokens import Keyword, DML, DDL, Punctuation, Whitespace, Newline, Name, String, Number, Operator

In [134]:
pp = pprint.PrettyPrinter(indent=4)

In [135]:
def get_fm_sql(file_path:str) -> str:

    '''
    Get formatted sql from file

    Parameters
    ----------
    file_path : str

    Returns
    -------
    indent_str : str
    '''

    with open(file_path, encoding='utf-8') as file:
        content = file.read()
        str_sql = sqlparse.format(content, reindent=True, keyword_case='upper')
        str_sql = str_sql.strip(' \t\n;')
    return str_sql

In [136]:
sql = get_fm_sql('tag000001.sql')

In [137]:
pp.pprint(sql)

("var x varchar(20) EXEC :x := to_char(sysdate, 'YYYYMMDDHH24MISS')\n"
 'INSERT INTO "1111111".TAG_EXECUTETIME(TRAN_ID, TAG_CD, STARTTIME, '
 'LOGINUSER)\n'
 'SELECT :x AS A,\n'
 '       1234567 AS TAG_CD,\n'
 '       SYSDATE AS STARTTIME,\n'
 '       USER AS LOGINUSER\n'
 'FROM DUAL\n'
 'COMMIT;\n'
 '\n'
 '\n'
 'SELECT 1234567 AS TAG_CD\n'
 'FROM DUAL;\n'
 '\n'
 '\n'
 'DROP TABLE TAG_2222222_TEMP_00;\n'
 '\n'
 '\n'
 'CREATE TABLE TAG_2222222_TEMP_00 AS\n'
 'SELECT CUST_ID,\n'
 '       FLG,\n'
 '       CNT,\n'
 '       TXN_AMT,\n'
 '       MAX_DT\n'
 'FROM\n'
 '  (SELECT 1,\n'
 '          2,\n'
 '          3,\n'
 '          4\n'
 '   FROM DSSDS.XXXXXXX A,\n'
 '        DSDEP.XXXXXXX B)\n'
 'GROUP BY CUST_ID;\n'
 '\n'
 '\n'
 'DROP TABLE TAG_3333333;\n'
 '\n'
 '\n'
 'CREATE TABLE TAG_3333333 AS\n'
 'SELECT CUST_ID,\n'
 '       FLG,\n'
 '       CNT,\n'
 '       TXN_AMT,\n'
 '       MAX_DT\n'
 'FROM TAG_4444444 A,\n'
 '\n'
 '  (SELECT X,\n'
 '          Y,\n'
 '          Z\n'
 '   FROM TAG_5

In [138]:
parse = sqlparse.parse(sql)

In [176]:
def extract_table(parse):
    '''
    Extract table from sqlparse.sql.Statement

    Parameters
    ----------
    parse : sqlparse.sql.Statement

    Returns
    -------
    table : list
    '''
    root_table = []
    inherit_table = []
    status = False
    loop = False
    root = False
    inherit = False
    m = 0


    for stmt in parse:

        L0_root = False
        inherit = False
        root_table = []
        inherit_table = []

        for token in stmt.tokens:
            if token._get_repr_name() == 'Keyword':
                if token.value == 'EXEC':
                    break
                elif token.value == 'FROM':
                    L0_root = True

                elif token.value == {'TABLE', 'INTO'}:
                    inherit = True
                    

            if inherit:

                if L0_root:

                    if (token._get_repr_name() in {'Identifier', 'IdentifierList', 'Parenthesis'}):

                        ### 以下建議抽象化

                        L1_root = False

                        ## if 字串內有from 進迴圈 else 加入root_table
                        # 1. 確認長度決定是否進迴圈
                        # 2. 抽取Table Name
                        # 3. Root False
                        for L1_token in token:

                            if L1_token._get_repr_name() == 'Identifier':
                                if (len(L1_token.value.split(' ')) == 2):
                                    root_table.append(L1_token.value.split(' ')[0])
                                
                                else:
                                    
                                    for L2_token in L1_token[0]:
                                        
                                        if (L2_token._get_repr_name() == 'Identifier'):
                                            root_table.append(L2_token.value)


                            if L1_token.value == 'FROM':
                                L1_root = True
                            
                            if L1_root:

                                if (L1_token._get_repr_name() in {'Identifier', 'IdentifierList', 'Parenthesis'}):
                                    for L2_token in L1_token:

                                        if (L2_token._get_repr_name() in {'Identifier', 'IdentifierList', 'Parenthesis'}):
                                            root_table.append(L2_token.value.split(' ')[0])
                
                else: 

                    if (token._get_repr_name() == 'Identifier'):
                        if len(inherit_table) == 0:
                            inherit_table.append(token.value)

        if (len(inherit_table) > 0) and (len(root_table) > 0): 
            pp.pprint('inherit')
            pp.pprint(inherit_table)
            pp.pprint('root')
            pp.pprint(root_table)

## 單筆僅繼承table的在進行drop
extract_table(parse)

In [177]:
test = '''
CREATE TABLE result_table (
SELECT 
    employees.employee_id,
    employees.first_name,
    employees.last_name,
    departments.department_name,
    salaries.salary_amount
FROM 
    employees
JOIN 
    departments ON employees.department_id = departments.department_id
JOIN 
    (SELECT 
         employee_id, MAX(salary_amount) as salary_amount
     FROM 
         salaries
     GROUP BY 
         employee_id) as salaries 
ON 
    employees.employee_id = salaries.employee_id
WHERE 
    employees.hire_date > 
    (SELECT 
         MIN(hire_date) 
     FROM 
         employees 
     WHERE 
         department_id = 'IT');'''

In [178]:
str_sql = sqlparse.format(test, reindent=True, keyword_case='upper')
test_parse = sqlparse.parse(str_sql)
extract_table(test_parse)

In [179]:
for stmt in test_parse:
    pp.pprint(stmt.tokens)

[   <Newline ' ' at 0x13E60A9A0>,
    <DDL 'CREATE' at 0x13E60AA00>,
    <Whitespace ' ' at 0x13E60AA60>,
    <Keyword 'TABLE' at 0x13E60AAC0>,
    <Whitespace ' ' at 0x13E60AB20>,
    <Identifier 'result...' at 0x13E60C9E0>,
    <Whitespace ' ' at 0x13E60ABE0>,
    <Punctuation '(' at 0x13E60AC40>,
    <Newline ' ' at 0x13E60ACA0>,
    <DML 'SELECT' at 0x13E60AD00>,
    <Whitespace ' ' at 0x13E60AD60>,
    <IdentifierList 'employ...' at 0x13E6174A0>,
    <Newline ' ' at 0x13E61A160>,
    <Keyword 'FROM' at 0x13E61A1C0>,
    <Whitespace ' ' at 0x13E61A220>,
    <Identifier 'employ...' at 0x13E60CC80>,
    <Newline ' ' at 0x13E61A2E0>,
    <Keyword 'JOIN' at 0x13E61A340>,
    <Whitespace ' ' at 0x13E61A3A0>,
    <Identifier 'depart...' at 0x13E60CEB0>,
    <Whitespace ' ' at 0x13E61A460>,
    <Keyword 'ON' at 0x13E61A4C0>,
    <Whitespace ' ' at 0x13E61A520>,
    <Comparison 'employ...' at 0x13E60CCF0>,
    <Newline ' ' at 0x13E61A9A0>,
    <Keyword 'JOIN' at 0x13E61AA00>,
    <Newline 

# Reference

In [34]:
import sqlparse
from sqlparse.sql import Parenthesis,Function,Identifier, IdentifierList
from sqlparse.tokens import Keyword, Name


COLUMN_OPERATIONS={'SELECT','FROM'}
FUNCTION_OPERATIONS={'SELECT','DROP','INSERT','UPDATE','CREATE'}
RESULT_OPERATIONS = {'UNION', 'INTERSECT', 'EXCEPT', 'SELECT'}
ON_KEYWORD = 'ON'
PRECEDES_TABLE_NAME = {'FROM', 'JOIN', 'DESC', 'DESCRIBE', 'WITH'}
global table_names
global column_names
global columns_rank
global function_names
global alias_names
table_names = []
column_names = []
function_names = []
alias_names = []
columns_rank = 0

In [36]:
def is_identifier(token):
    return isinstance(token, (IdentifierList, Identifier))


def is_identifiers(token):
    return isinstance(token, Identifier)


def is_identifiersList(token):
    return isinstance(token, IdentifierList)


def is_Function(token):
    return isinstance(token, Function)


def is_Parenthesis(token):
    return isinstance(token, Parenthesis)


def precedes_function_name(token_value):
    for keyword in FUNCTION_OPERATIONS:
        if keyword in token_value:
            return True
    return False

In [ ]:
# 获得该SQL主要功能函数
def get_main_functionsql(statment):
    return statment.get_type()

# 第一层Identifier
def _get_one_Identifier(statment):
    idfr_list = []
    tokens_list = statment.tokens
    for each_token in tokens_list:
        if each_token._get_repr_name() == 'Identifier':
            idfr_list.append(each_token)
    return idfr_list


In [ ]:
# 满足库.表形式的identifiers提取即判定为表
def get_Identifier_keywords_tables(identifiers):
    global table_names
    if len(identifiers.tokens) == 3 and identifiers.tokens[1].value == ' ':
        a = identifiers.tokens[0].value
        return table_names.append(a)
    if len(identifiers.tokens) > 1 and identifiers.tokens[1].value == '.':
        a = identifiers.tokens[0].value
        b = identifiers.tokens[2].value
        db_table = (a, b)
        full_tree = '{}.{}'.format(a, b)
        if len(identifiers.tokens) == 3:
            return table_names.append(full_tree)
        else:
            i = identifiers.tokens[3].value
            c = identifiers.tokens[4].value
            if i == ' ':
                return table_names.append(full_tree)
            full_tree = '{}.{}.{}'.format(a, b, c)
            return table_names.append(full_tree)
    if len(identifiers.tokens) == 1:
        a = identifiers.tokens[0].value
        return table_names.append(a)

In [ ]:
def precedes_table_name(token_value):
    for keyword in PRECEDES_TABLE_NAME:
        if keyword in token_value:
            return True
    return False

In [ ]:
# 提底表表名
def _extract_table_from_token(statment):
    if not hasattr(statment, 'tokens'):
        return
    # 可添加多个preceding_token
    table_name_preceding_token = False

    for item in statment.tokens:
        # 除Identifier/IdentifierList以外还有Parenthesis和Function有group，这些需要递归
        if item.is_group and not is_identifier(item):
            _extract_table_from_token(item)

        # 启动函数，依赖PRECEDES_TABLE_NAME，当为指定Keyword时候启发table_name_preceding_token
        if item.ttype in Keyword:
            # 有关键字的情况下可以判定存在表，那么直接跳到符合的情况下，剩余的token不再判断
            if precedes_table_name(item.value.upper()):
                table_name_preceding_token = True
                continue
                # 那么直接跳到符合的情况下，剩余的token不再判断
        if not table_name_preceding_token:
            continue
        # 可能From里面也嵌套查询等外表，那么再次设定为False再判断一次
        if item.ttype in Keyword or item.value == ',':
            if (is_result_operation(item.value) or
                    item.value.upper() == ON_KEYWORD):
                table_name_preceding_token = False
                continue
            # FROM clause is over
            break

        # 只有identifiers和IdentifierList会有库.表
        if isinstance(item, Identifier):
            process_identifier(item)

        if isinstance(item, IdentifierList):
            for token in item.tokens:
                if is_identifier(token):
                    process_identifier(token)

                # 该方法解析IdentifierList


In [ ]:
# 表血缘
def blood_table(statment):
    if precedes_function_name(get_main_functionsql(statment)):
        idfr_list = get_one_Identifier(statment)
        get_Identifier_keywords_tables(idfr_list[0])
    type_name = get_main_functionsql(statment)
    extract_table_from_token(statment)
    inherit_table = table_names[0]
    root_table = set(table_names[1:])
    if get_main_functionsql(statment) != 'SELECT':
        table_Bloodcurse = '{}->{}'.format(inherit_table, root_table)
        return table_Bloodcurse
    else:
        table_Bloodcurse = set(table_names)
        return table_Bloodcurse